In [ ]:
import  chargeback_rpt.monthly_report_builder as reporter

import chargeback_rpt.vm_data_utility as vm_util
import chargeback_rpt.file_directory_manager as fd_mn
import time
import datetime
import sys 
import os


In [ ]:
run_py=True
fix_mmyy=False  #fix_mmyy=True  for test_only_jupyternotebook
no_argv=1  # 1 for run wind scheduler
is_monthly=True  # this version support only montly report

x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
month_x=str(int(month_x))
year_x=x_datenow.strftime('%Y')



if fix_mmyy:
 month_param='6'
 year_param='2021'
else:
 month_param=month_x
 year_param=year_x
    

sr_bill_info=None
print(f"Current month-year : {month_param}-{year_param}")

In [ ]:
def enter_bill_info(bill_info_sr):
  
 try:
  
  print("Enter date for billing  or press enter if skip")   
  # input new one to replace initial value 
  msg_text="Enter {} date (dd-mm-YYYY , ex {}) : "

  myBill_from_str = input(msg_text.format("Bill-From",'such as 26-12-2020') or '')
  if  myBill_from_str!='':
    bill_info_sr['bill_from']=datetime.datetime.strptime(myBill_from_str, '%d-%m-%Y')
       
  myBill_to_str = input(msg_text.format("Bill-To",'such as25-1-2021') or '')
  if  myBill_to_str!='':
    bill_info_sr['bill_to']=datetime.datetime.strptime(myBill_to_str, '%d-%m-%Y')
    

  myInvoice_str = input(msg_text.format("Invoice",'such as 31-1-2021') or '')
  if  myInvoice_str!='':
    bill_info_sr['bill_invoice']=datetime.datetime.strptime(myInvoice_str, '%d-%m-%Y')
    
    
  myDue_str = input(msg_text.format("Due",'such as 28-2-2021') or '')
  if  myDue_str!='':
    bill_info_sr['bill_due']=datetime.datetime.strptime(myDue_str, '%d-%m-%Y')  
    
  return bill_info_sr        

 except Exception as ex:
  print(ex)
          
  

In [ ]:
def check_existing_report():
    
#print("Select report building type (press 1 or 2)")
#type_param = int(input("Press 1=montly report OR Press 2=ac-hoc report  : ") or "1") 
 type_param=1
 is_replaced=True

 if type_param ==1:
   is_monthly=True
 elif  type_param ==2:
   is_monthly=False
 else:
   raise Exception("Report building type can be either 1 or 2")
    
 if is_monthly==True :

   report_path=os.path.join(vm_util.get_config_value('report_rootpath',None),f'{month_param}-{year_param}')
   x=fd_mn.is_notEmpty_folder(report_path)
    
   if x>0:
    
    print("Do you want to replace existing report  with new ones?")
    print(f"{report_path} exists")
    press_y2=input(f"Press y=True OR anykeys=False : ")
    if press_y2.lower()=='y':
     is_replaced=True
    else:
     is_replaced=False
 
 else:
     is_replaced=True
        
 return is_replaced

In [ ]:
try:

    if  run_py:

        press_y='n'
        ok=False
        ok2=False

        # paramater for running with job scheduler
        if len(sys.argv) > no_argv:
            if sys.argv[1]=='0'  and sys.argv[2]=='0' and sys.argv[3]=='0':

             month_param=month_x
             year_param=year_x 
             sr_bill_info=None
             is_monthly=True
             ok=True  
             ok2=True 

            else:
             raise Exception("Specify 0 0 as arguments")   

        # paramater for running as manual    
        else:
            print("Enter month and year for the report , or press enter if current month or year")   
            month_param = int(input("Enter the month (1-12) : ") or month_x)
            year_param = int(input("Enter the year(ex 2021) : ") or year_x)

#             print(f"Do you want to enter bill info manually")
#             press_bill=input(f"Press y=True  OR anykeys=False : ") 
            
#             sr_bill_info=vm_util.get_initial_bill_info(int(month_param),int(year_param),None) 
#             if press_bill.lower()=='y':
#              sr_bill_info =enter_bill_info(sr_bill_info)
               

            ok2 = check_existing_report()
            
            print("=============================================================================")  
            print(f"Build report for ")
            
            report_dt= datetime.datetime(int(year_param), int(month_param), 1)
            print(f"month={report_dt.strftime('%B')} and year={report_dt.strftime('%Y')}")
#             print(f"bill from : {sr_bill_info['bill_from'].strftime('%d-%m-%Y')} to {sr_bill_info['bill_to'].strftime('%d-%m-%Y')}")
#             print(f"invoice date : {sr_bill_info['bill_invoice'].strftime('%d-%m-%Y')}")
#             print(f"due date : {sr_bill_info['bill_due'].strftime('%d-%m-%Y')}")
            print("=============================================================================")  
                        
            press_y=input(f"Confirm Build report ==> press y : ") 
            if press_y.lower()=='y':
             ok=True


        if ok==True and ok2==True:
            t_Start=time.time()
            print(datetime.datetime.now())

            reporter.create_monthly_report(month_param,year_param,sr_bill_info,is_monthly)

            print(datetime.datetime.now())
            t_End=time.time()
            t_elapsed=(t_End-t_Start)/60/60
            print('Total time to buid report : ',round(t_elapsed,2))
            
        else:
             quit()
    

except Exception as ex:
    print(ex)   
    raise ex


<!-- t_Start=time.time()
print(datetime.datetime.now())
#reporter.create_monthly_report(month_param,year_param,is_monthly)
print(datetime.datetime.now())
t_End=time.time()
t_elapsed=(t_End-t_Start)/60/60
print('Total execute making report : ',round(t_elapsed,2)) -->